In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

df = pd.read_csv('SF_industrial_leases_2021.csv')
df.drop('#', axis=1, inplace=True)

In [14]:
df.columns = df.columns.str.replace('(','')
df.columns = df.columns.str.replace(')','')
df.columns = df.columns.str.replace(' ','_')
#df.to_csv('2021_industrial_leases_for_graphic')
df['Size_SF'] = df['Size_SF'].replace(',','',regex=True)
df['Size_SF'] = pd.to_numeric(df['Size_SF'])
df.dtypes

<ipython-input-14-285824d8e123>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('(','')
<ipython-input-14-285824d8e123>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(')','')


Business_Park    object
Address          object
City             object
Property_Type    object
Tenant_Name      object
Size_SF           int64
Lease_Type       object
Sign_Date        object
Submarket        object
Clips            object
dtype: object

In [21]:
df['geo_address'] = df['Address'] + ' ' + df['City'] + ' FL'
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df

,Business_Park,Address,City,Property_Type,Tenant_Name,Size_SF,Lease_Type,Sign_Date,Submarket,Clips,geo_address
0,First Park Miami,8801 NW 87th Ave,Miami,Warehouse,Carbel,368287,New,Jul-21,Medley/Hialeah Gardens,https://commercialobserver.com/2021/07/carbel-...,8801 NW 87th Ave Miami FL
1,Bridge Point Commerce Center,4310 NW 215th St,Miami Gardens,Warehouse,City Furniture,286875,New,Mar-21,Northeast Dade,https://therealdeal.com/miami/2021/03/31/city-...,4310 NW 215th St Miami Gardens FL
2,3750 Palm Dr,3750 Palm Dr,Homestead,Warehouse,Amazon,216000,New,Mar-21,Outlying Miami-Dade,https://therealdeal.com/miami/2020/09/16/homes...,3750 Palm Dr Homestead FL
3,Coral Springs Distribution Center I Bldg C,4030 NW 126th Ave,Coral Springs,Warehouse,Amazon,215486,New,May-21,Northwest Broward,none,4030 NW 126th Ave Coral Springs FL
4,International Corporate Park,10205 NW 19th St,Miami,Warehouse,Expeditors International,206000,Renewal,Feb-21,Miami Airport West,none,10205 NW 19th St Miami FL
5,Coral Springs Commerce Center II Bldg 4,4250 Coral Ridge Dr,Coral Springs,Warehouse,Ferguson Enterprises,194441,New,Aug-21,Northwest Broward,none,4250 Coral Ridge Dr Coral Springs FL
6,3350-3450 NW 112th St,3350-3450 NW 112th St,Miami,Warehouse,Retrievex / Access Information Management,187749,Renewal,Q1 2021,Northeast Dade,none,3350-3450 NW 112th St Miami FL
7,Bridge Point Commerce Center Bldg B,21201 NW 43rd Ave,Miami Gardens,Warehouse,USPS,179634,New,Jul-21,Northeast Dade,none,21201 NW 43rd Ave Miami Gardens FL
8,Tamarac Business Center Bldg I,6901 N Hiatus Rd,Tamarac,Warehouse,Amazon,178322,New,Apr-21,Southwest Broward,https://tamaractalk.com/amazon-to-open-fulfill...,6901 N Hiatus Rd Tamarac FL
9,Prologis Beacon Lakes 24,13155 NW 19th Ln,Miami,Warehouse,King Ocean,157258,New,Dec-21,Miami Airport West,https://therealdeal.com/miami/2021/12/06/lease...,13155 NW 19th Ln Miami FL


In [121]:
#df.at[0,'Property Address']=('1111 North Ocean Drive, Hollywood, FL')

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [22]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [23]:
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)

In [24]:
df.head(1)

,Business_Park,Address,City,Property_Type,Tenant_Name,Size_SF,Lease_Type,Sign_Date,Submarket,Clips,geo_address,loc
0,First Park Miami,8801 NW 87th Ave,Miami,Warehouse,Carbel,368287,New,Jul-21,Medley/Hialeah Gardens,https://commercialobserver.com/2021/07/carbel-...,8801 NW 87th Ave Miami FL,"(8801 NW 87th Ave, Miami, FL 33166, USA, (25.8..."


In [25]:
df['loc']

0    (8801 NW 87th Ave, Miami, FL 33166, USA, (25.8...
1    (4310 NW 215th St, Miami Gardens, FL 33055, US...
2    (3750 Palm Dr, Homestead, FL 33035, USA, (25.4...
3    (4030 NW 126th Ave, Coral Springs, FL 33076, U...
4    (10205 NW 19th St, Doral, FL 33172, USA, (25.7...
5    (4250 Coral Ridge Dr, Coral Springs, FL 33076,...
6    (3350 NW 112th St, Miami, FL 33167, USA, (25.8...
7    (21201 NW 43rd Ave, Miami Gardens, FL 33055, U...
8    (6901 Hiatus Rd, Tamarac, FL 33321, USA, (26.2...
9    (NW 19th Ln, Miami, FL 33136, USA, (25.7947119...
Name: loc, dtype: object

In [26]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [27]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,Business_Park,Address,City,Property_Type,Tenant_Name,Size_SF,Lease_Type,Sign_Date,Submarket,Clips,geo_address,loc,point,lat,lon,altitude
0,First Park Miami,8801 NW 87th Ave,Miami,Warehouse,Carbel,368287,New,Jul-21,Medley/Hialeah Gardens,https://commercialobserver.com/2021/07/carbel-...,8801 NW 87th Ave Miami FL,"(8801 NW 87th Ave, Miami, FL 33166, USA, (25.8...","(25.840994, -80.33831789999999, 0.0)",25.840994,-80.338318,0.0
1,Bridge Point Commerce Center,4310 NW 215th St,Miami Gardens,Warehouse,City Furniture,286875,New,Mar-21,Northeast Dade,https://therealdeal.com/miami/2021/03/31/city-...,4310 NW 215th St Miami Gardens FL,"(4310 NW 215th St, Miami Gardens, FL 33055, US...","(25.9696633, -80.27115979999999, 0.0)",25.969663,-80.271160,0.0
2,3750 Palm Dr,3750 Palm Dr,Homestead,Warehouse,Amazon,216000,New,Mar-21,Outlying Miami-Dade,https://therealdeal.com/miami/2020/09/16/homes...,3750 Palm Dr Homestead FL,"(3750 Palm Dr, Homestead, FL 33035, USA, (25.4...","(25.4467308, -80.42246159999999, 0.0)",25.446731,-80.422462,0.0
3,Coral Springs Distribution Center I Bldg C,4030 NW 126th Ave,Coral Springs,Warehouse,Amazon,215486,New,May-21,Northwest Broward,none,4030 NW 126th Ave Coral Springs FL,"(4030 NW 126th Ave, Coral Springs, FL 33076, U...","(26.283099, -80.2938057, 0.0)",26.283099,-80.293806,0.0
4,International Corporate Park,10205 NW 19th St,Miami,Warehouse,Expeditors International,206000,Renewal,Feb-21,Miami Airport West,none,10205 NW 19th St Miami FL,"(10205 NW 19th St, Doral, FL 33172, USA, (25.7...","(25.7912463, -80.3613988, 0.0)",25.791246,-80.361399,0.0
5,Coral Springs Commerce Center II Bldg 4,4250 Coral Ridge Dr,Coral Springs,Warehouse,Ferguson Enterprises,194441,New,Aug-21,Northwest Broward,none,4250 Coral Ridge Dr Coral Springs FL,"(4250 Coral Ridge Dr, Coral Springs, FL 33076,...","(26.2848911, -80.2842221, 0.0)",26.284891,-80.284222,0.0
6,3350-3450 NW 112th St,3350-3450 NW 112th St,Miami,Warehouse,Retrievex / Access Information Management,187749,Renewal,Q1 2021,Northeast Dade,none,3350-3450 NW 112th St Miami FL,"(3350 NW 112th St, Miami, FL 33167, USA, (25.8...","(25.875701, -80.25398299999999, 0.0)",25.875701,-80.253983,0.0
7,Bridge Point Commerce Center Bldg B,21201 NW 43rd Ave,Miami Gardens,Warehouse,USPS,179634,New,Jul-21,Northeast Dade,none,21201 NW 43rd Ave Miami Gardens FL,"(21201 NW 43rd Ave, Miami Gardens, FL 33055, U...","(25.9640048, -80.2715303, 0.0)",25.964005,-80.271530,0.0
8,Tamarac Business Center Bldg I,6901 N Hiatus Rd,Tamarac,Warehouse,Amazon,178322,New,Apr-21,Southwest Broward,https://tamaractalk.com/amazon-to-open-fulfill...,6901 N Hiatus Rd Tamarac FL,"(6901 Hiatus Rd, Tamarac, FL 33321, USA, (26.2...","(26.2082152, -80.2911445, 0.0)",26.208215,-80.291145,0.0
9,Prologis Beacon Lakes 24,13155 NW 19th Ln,Miami,Warehouse,King Ocean,157258,New,Dec-21,Miami Airport West,https://therealdeal.com/miami/2021/12/06/lease...,13155 NW 19th Ln Miami FL,"(NW 19th Ln, Miami, FL 33136, USA, (25.7947119...","(25.7947119, -80.20136959999999, 0.0)",25.794712,-80.201370,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [28]:
df.columns

Index(['Business_Park', 'Address', 'City', 'Property_Type', 'Tenant_Name',
       'Size_SF', 'Lease_Type', 'Sign_Date', 'Submarket', 'Clips',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [35]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=8)
# folium.TileLayer('stamentoner').add_to(m)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'2021 South Florida Top Industrial Leases')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('Amazon made the list twice.')


marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Business_Park"], r["Address"], r["City"], r["Property_Type"], r["Tenant_Name"], r["Size_SF"], r["Lease_Type"], r["Sign_Date"], r["Submarket"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Business Park: {bold_start}{info[0]}{bold_end}{new_line}\
    Address: {bold_start}{info[1]}{bold_end}{new_line}\
    City: {bold_start}{info[2]}{bold_end}{new_line}\
    Property Type: {bold_start}{info[3]}{bold_end}{new_line}\
    Tenant Name: {bold_start}{info[4]}{bold_end}{new_line}\
    Size SF: {bold_start}{info[5]}{bold_end}{new_line}\
    Lease Type: {bold_start}{info[6]}{bold_end}{new_line}\
    Sign Date: {bold_start}{info[7]}{bold_end}{new_line}\
    Submarket: {bold_start}{info[8]}{bold_end}{new_line}'
    
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
m

In [34]:
m.save('index.html')

In [37]:
df.to_csv('2021_SF_industrial_leases_for_graphic.csv')